In [1]:
from tg.common import DataBundle

db = DataBundle.load('files/tsa-test.zip')

In [2]:
from tg.common.ml import batched_training as bt

idb = bt.IndexedDataBundle(db.index.loc[db.index.split=='train'].sample(5, random_state=45), db)
idb.index_frame

,word_id,sentence_id,label,split
sample_id,,,,
26041,26041,26040,0,train
179036,179036,178987,0,train
148318,148318,148172,1,train
39578,39578,39509,1,train
50058,50058,49904,1,train


In [3]:
from tg.grammar_ru import Separator
import numpy as np

df = idb.bundle.src
df = df.loc[df.sentence_id.isin(idb.index_frame.sentence_id)].copy()
df = df.merge(
    idb.index_frame.reset_index().set_index('sentence_id')[['sample_id']], 
    left_on='sentence_id', 
    right_index=True
)
df['mark'] = np.where(df.is_target, df.label.astype(str), 'no')

Separator.Viewer().tooltip('sample_id').highlight('mark', {'0':'#ff0000', '1':'#00ff00'}).to_html_display(df)

In [4]:
from tg.common.ml import dft

extractor = (bt
             .PlainExtractor
             .build('features')
             .index()
             .join('pymorphy', on_columns='word_id')
             .apply(
                 transformer=dft.DataFrameTransformerFactory.default_factory(),
                 drop_columns=['normal_form', 'alternatives']
             ))
extractor.fit_extract(idb)

,score,delta_score,POS_INFN,POS_VERB,animacy_NULL,gender_NULL,number_sing,number_NULL,case_NULL,aspect_impf,aspect_perf,transitivity_intr,person_3per,person_NULL,tense_pres,tense_NULL,mood_indc,mood_NULL,voice_NULL,involvement_NULL
sample_id,,,,,,,,,,,,,,,,,,,,
26041,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0
179036,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0
148318,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0
39578,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0
50058,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0


In [5]:
from tg.grammar_ru.components import CoreExtractor

extractor = CoreExtractor('features')
extractor.fit_extract(idb)

2023-01-27 15:27:13.819788 INFO: Fitting extractor pymorphy in CoreExtractor
2023-01-27 15:27:13.945134 INFO: Success
2023-01-27 15:27:13.950223 INFO: Fitting extractor slovnet_morph in CoreExtractor
2023-01-27 15:27:13.957528 INFO: Skipped as the corresponding frame is missing from the bundle
2023-01-27 15:27:13.963302 INFO: Fitting extractor slovnet_syntax in CoreExtractor
2023-01-27 15:27:13.964639 INFO: Skipped as the corresponding frame is missing from the bundle
2023-01-27 15:27:13.965921 INFO: Fitting extractor syntax_fixes in CoreExtractor
2023-01-27 15:27:13.969115 INFO: Skipped as the corresponding frame is missing from the bundle
2023-01-27 15:27:13.972070 INFO: Fitting extractor syntax_stats in CoreExtractor
2023-01-27 15:27:13.974435 INFO: Skipped as the corresponding frame is missing from the bundle


,pymorphy_score,pymorphy_delta_score,pymorphy_alternatives,pymorphy_POS_INFN,pymorphy_POS_VERB,pymorphy_animacy_NULL,pymorphy_gender_NULL,pymorphy_number_sing,pymorphy_number_NULL,pymorphy_case_NULL,...,pymorphy_aspect_perf,pymorphy_transitivity_intr,pymorphy_person_3per,pymorphy_person_NULL,pymorphy_tense_pres,pymorphy_tense_NULL,pymorphy_mood_indc,pymorphy_mood_NULL,pymorphy_voice_NULL,pymorphy_involvement_NULL
sample_id,,,,,,,,,,,,,,,,,,,,,
26041,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,...,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0
179036,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,...,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0
148318,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0
39578,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0
50058,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,...,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0


In [6]:
from tg.grammar_ru.components import PlainContextBuilder

context_builder = PlainContextBuilder(
    include_zero_offset=True,
    left_to_right_contexts_proportion=0.5
)

context_builder.build_context(idb, 5).sort_index()

another_word_id
sample_id offset                 
26041     -1                26040
           0                26041
           1                26042
           2                26043
39578     -2                39576
          -1                39577
           0                39578
           1                39579
           2                39580
50058     -1                50057
           0                50058
           1                50059
           2                50060
148318    -2               148316
          -1               148317
           0               148318
           1               148319
           2               148320
179036    -2               179034
          -1               179035
           0               179036
           1               179037
           2               179038